1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
   - 토큰 수 초과로 답변을 생성하지 못할 수 있음
   - 문서가 길면 (input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [1]:
# DOCX 파일을 로드하기 위한 로더 생성
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# tax.docx 파일을 읽을 로더 객체 생성
loader = Docx2txtLoader("./tax.docx")

# 문서 로드 (Document 객체 리스트 반환)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()  # .env 파일에서 환경 변수 로드

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name="chroma-tax",
#     persist_directory="./chroma_db",
# )

database = Chroma(
    collection_name="chroma-tax",
    persist_directory="./chroma_db",
    embedding_function=embedding,
)


In [6]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 유사도 검색 테스트
retrieved_docs = database.similarity_search(query, k=4)

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [8]:
prompt = f"""
[Identity]

- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [9]:
ai_message = llm.invoke(prompt)

ai_message

AIMessage(content='한국 소득세 계산은 여러 가지 변수에 따라 달라질 수 있는데, 기본적인 계산 방법을 알아보겠습니다. 연봉 5천만 원인 직장인의 소득세는 다음과 같이 계산할 수 있습니다:\n\n1. **기본 소득세율**: 한국의 소득세는 누진세율 구조를 가지고 있습니다. 일반적으로 소득 구간에 따라 다른 세율이 적용됩니다. 연봉 5천만 원이 적용될 수 있는 기본 세율은 아래와 같습니다(2023년 기준, 실제 적용 세율은 변동 가능):\n\n   - 1,200만 원 이하: 6%\n   - 1,200만 원 초과 ~ 4,600만 원 이내: 15%\n   - 4,600만 원 초과 ~ 8,800만 원 이내: 24%\n   - 이후 구간에 따라 더 높은 세율이 적용됩니다.\n\n2. **세액 공제 및 감면**: 기본 세율에 따른 세액을 계산한 후에는 각종 공제와 감면을 적용해야 합니다. 이는 연말정산 시 다양한 요인(공제항목 등)에 따라 차이가 있습니다.\n\n대략적인 소득세 계산 예시는 다음과 같습니다.\n\n- 첫 1,200만 원에 대해: 1,200만 원 * 6% = 72만 원\n- 다음 3,400만 원(4,600 - 1,200만 원)에 대해: 3,400만 원 * 15% = 510만 원\n- 남은 400만 원(5,000 - 4,600만 원)에 대해: 400만 원 * 24% = 96만 원\n\n총 세액은 72만 원 + 510만 원 + 96만 원 = 약 678만 원이 됩니다.\n\n이 총 세액에서 공제 및 감면 항목들을 적용하여 최종 납부할 소득세를 결정해야 합니다. 실제 세액 계산은 개인의 상황에 따라 다르게 적용될 수 있으므로, 정확한 세액은 국세청 연말정산 서비스나 세무 전문가의 도움을 받아 확인하는 것을 권장드립니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 470, 'prompt_tokens': 4024, 'total_